## Prompt Templates and Output Parsers

In [4]:
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a.env file
_ = load_dotenv(find_dotenv())

# openai_api_key = os.environ["OPENAI_API_KEY"]

# Load opensource api models
groql_api_key = os.environ["GROQ_API_KEY"]

# Completion Models

In [5]:
# working with Groq models

from langchain_groq import ChatGroq

llamaChatModel = ChatGroq(
    model="llama3-70b-8192"
)

## Prompts and Prompt Templates

In [7]:
# import prompt templates from langchain

# This is for completions model
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} story about {topic}"
)

# How to parse the input strings to 
llmModelPrompt = prompt_template.format(
    adjective="curious",
    topic="Kennedy Family"
)

# call the model
res = llamaChatModel.invoke(llmModelPrompt)
print(res)

content='The Kennedy family! So much history, drama, and intrigue surrounding this American royal family. Here\'s a curious story that might pique your interest:\n\n**The Mysterious Case of the Kennedy Family\'s "Wild Child"**\n\nMeet Rosemary Kennedy, the third child and first daughter of Joseph P. Kennedy Sr. and Rose Fitzgerald Kennedy. Born in 1918, Rosemary was known for her beauty, athleticism, and lively personality. However, she struggled with learning disabilities and behavioral issues from an early age, which frustrated her ambitious father and led to a series of unconventional and, some might say, disturbing decisions.\n\nIn the 1940s, Rosemary\'s behavior became increasingly erratic, and her parents sought solutions to "cure" her of her perceived problems. They consulted with various doctors and eventually settled on a radical treatment: a prefrontal lobotomy, a procedure that involved severing nerve connections in the brain to calm the patient\'s emotions.\n\nOn November 7

In [8]:
# print chunked response
print(res.content)

The Kennedy family! So much history, drama, and intrigue surrounding this American royal family. Here's a curious story that might pique your interest:

**The Mysterious Case of the Kennedy Family's "Wild Child"**

Meet Rosemary Kennedy, the third child and first daughter of Joseph P. Kennedy Sr. and Rose Fitzgerald Kennedy. Born in 1918, Rosemary was known for her beauty, athleticism, and lively personality. However, she struggled with learning disabilities and behavioral issues from an early age, which frustrated her ambitious father and led to a series of unconventional and, some might say, disturbing decisions.

In the 1940s, Rosemary's behavior became increasingly erratic, and her parents sought solutions to "cure" her of her perceived problems. They consulted with various doctors and eventually settled on a radical treatment: a prefrontal lobotomy, a procedure that involved severing nerve connections in the brain to calm the patient's emotions.

On November 7, 1941, at the age of

## Using the Chat Completions Template
You can parse two prompt templates:
- the "system" prompt
- the "human" prompt

In [11]:
# chat completion model

from langchain_core.prompts import ChatPromptTemplate

# Prompt Engineering  - Zero shot prompting
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an {professional} expert on {topic}."),
        ("human", "Hello Mr. {professional}, can you please answer a question"),
        ("ai", "sure!"),
        ("human", "{user_input}")
    ]
)

# Parse variables to the template
messages = chat_template.format_messages(
    professional="Historian", 
    topic="The Kennedy Family",
    user_input="How many grandchildren had Joseph P. Kennedy?"
)

# call the chatModel
response = llamaChatModel.invoke(messages)

# print the response
print(response.content)


Joseph P. Kennedy, the patriarch of the Kennedy family, had a total of 29 grandchildren. His nine children, including John F. Kennedy, Robert F. Kennedy, Ted Kennedy, and their siblings, went on to have a large brood of children of their own. In fact, the Kennedy family is known for its large and close-knit family, with many family members remaining involved in politics, public service, and philanthropy to this day.


# Old way of Templating

In [12]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are an Historian expert on the Kennedy family."
            )
        ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_template.format_messages(
    user_input="Name the children and grandchildren of Joseph P. Kennedy?"
)

response = llamaChatModel.invoke(messages)

In [14]:
# Print Template
print(response.content)

Joseph P. Kennedy Sr., the patriarch of the Kennedy family, had nine children with his wife Rose Fitzgerald Kennedy. Here are their names:

1. Joseph P. Kennedy Jr. (1915-1944)
2. John F. Kennedy (1917-1963)
3. Rosemary Kennedy (1918-2005)
4. Kathleen Kennedy (1920-1948)
5. Eunice Kennedy (1921-2009)
6. Patricia Kennedy (1924-2006)
7. Robert F. Kennedy (1925-1968)
8. Jean Kennedy (born 1928)
9. Edward M. Kennedy (1932-2009)

As for the grandchildren of Joseph P. Kennedy Sr., there are a total of 29 grandchildren from his nine children. Here are their names:

**Children of Joseph P. Kennedy Jr.**

* None (Joseph P. Kennedy Jr. died in 1944, before having children)

**Children of John F. Kennedy**

1. Arabella Kennedy (1956-1956) (stillborn)
2. Caroline Bouvier Kennedy (born 1957)
3. John Fitzgerald Kennedy Jr. (1960-1999)

**Children of Rosemary Kennedy**

* None (Rosemary Kennedy had intellectual disabilities and was institutionalized for most of her life)

**Children of Kathleen Kenne

## Few Shot Prompting
You always have to give an example

In [16]:
# import 
from langchain_core.prompts import FewShotChatMessagePromptTemplate

# language translator from english to sppanish
# you give english text and output: is spanish
examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

# 2. create template for example
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 3. Prepare the few shot template For the example_prompt
few_shot_prompt = FewShotChatMessagePromptTemplate(
       example_prompt=example_prompt,
       examples=examples
)

# Create final chat prompt template
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an English-Spanish translator"),
        few_shot_prompt,        # pasrse few shot template
        ("human", "{input}"),   # user input passed
    ]
)


### Chains

It's more of a pipeline for LLM execution.
- Prompt --> LLM --> Executable output
- Link: https://python.langchain.com/v0.1/docs/modules/chains/

In [17]:
# chains --> Pipe symbol
chain = final_prompt | llamaChatModel

# Invoke chain pass dictionary inputs
res = chain.invoke({"input":"How are you?"})

print(res.content)


Estoy bien, ¿y tú?


# Langchain Expressions Language

## Output Parser

In [21]:
# using a Completions Model
from langchain_openai import OpenAI

llmModel = OpenAI()

In [23]:
from langchain_core.prompts import PromptTemplate
# For chat model using a "ChatPromptTemplate"
from langchain.output_parsers.json import SimpleJsonOutputParser

# return a json output
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an 'answer' key that answers the following question: {question}"
)

# create a parser object to be my output
json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmModel | json_parser   # using a Completions Model


In [26]:
# Invoke
res = json_chain.invoke({"question":"What is the biggest country?"})
res

{'answer': 'Russia'}

In [27]:
# check the datatype
type(res)

dict

### Optionally, you can use Pydantic to define a custom output format

In [28]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

In [31]:
# Define a Pydantic Object with the desired output format
class Joke(BaseModel):
    setup:str = Field(description="question to set up a joke")
    punchline:str = Field(description="answer to resolve the joke")

In [35]:
# Example of format instructions
json_parser.get_format_instructions()

'Return a JSON object.'

In [37]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt definition.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Create a chain with the prompt and the parser
chain = prompt | llmModel | parser

# Invoke the chain
chain.invoke({"query":"Tell me a joke"})


{'setup': 'Why did the tomato turn red?',
 'punchline': 'Because it saw the salad dressing!'}